In [ ]:
import pandas as pd
import numpy as np
import loader as load
import config
import matplotlib
import matplotlib.pyplot as plt

files = ['aak_ge', 'tcma_gen', 'tcma_gen_aak_ge']
c_data_features = [pd.read_csv(fr"Data\Descriptor\Prediction_Tables\random_sampling\stage\{x}_linreg_predfeature.csv", index_col=None) for x in files]

In [ ]:
overlap = c_data_features[-1]
overlap

In [ ]:
# p: 200, target: tumor/stage, selection: linreg, layer: overlap
# what are the most selected bacterial features per cancer and across cancers
# what are the most selected Microbial features per cancer and across cancers
x = "tcma_gen_aak_ge"
p = 200
c = "STAD"
all_features, _ = load.getFeatures()
tcma_gen_features, aak_ge_features = all_features
    
for target in config.prediction_targets[:1]:
    for selection_type in config.selection_types[:1]:
        data_features = pd.read_csv(fr"Data\Descriptor\Prediction_Tables\random_sampling\{target}\{x}_{selection_type}_predfeature.csv", index_col=None)
        data_features = data_features[data_features.p == p]
        data_features = data_features[data_features.cancer == c]
        feature_frequencies = data_features["features"].value_counts()
        
        print(feature_frequencies)

        cell_text = feature_frequencies.values.reshape(980, 1)

        # plt.figure(linewidth=2,
        #    edgecolor=fig_border,
        #    facecolor=fig_background_color,
        #    tight_layout={'pad':1},
           #figsize=(5,3)
        #   )
        # Add a table at the bottom of the axes
        the_table = plt.table(cellText=cell_text,
                            rowLabels=feature_frequencies.index,
                            #   rowColours=rcolors,
                            #   rowLoc='right',
                            #   colColours=ccolors,
                            #   colLabels=column_headers,
                            loc='center')
        plt.show()